In [ ]:
pip install yfinance

In [ ]:
# -*- coding: utf-8 -*-

'''
#################### CONNECT TO THE FOLDER ####################
'''
def connect_to_the_folder(method):
    import os
    if method == 'google':
        from google.colab import drive
        drive.mount('/content/drive/', force_remount=True)
        os.chdir('/content/drive/My Drive/Data_Technology/DT_Learning/III_Big_Data/BDSE21_Team1/Final_Project/')
    elif method == 'local':
        os.chdir('c:\Shared\BDSE_Team1\Final_Project\\')

connect_to_the_folder('google') # connect to google or local

import yfinance as yf
from datetime import date, timedelta
today = date.today()
start = today - timedelta(days=20)
today = today.strftime('%Y-%m-%d')
start = start.strftime('%Y-%m-%d')

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
model_1 = load_model('Model/Model_Save/rnn_close_1_1.h5')
model_2 = load_model('Model/Model_Save/rnn_close_2_1.h5')
model_3 = load_model('Model/Model_Save/rnn_close_3_1.h5')
model_4 = load_model('Model/Model_Save/rnn_close_4_1.h5')
model_5 = load_model('Model/Model_Save/rnn_close_5_1.h5')
models = [model_1, model_2, model_3, model_4, model_5]

def get_data(code):
    # get realtime data
    df = yf.Ticker(str(code)+'.TW').history(start=start, end=today)
    if len(df) == 0:
        df = yf.Ticker(str(code)+'.TWO').history(start=start, end=today)
        if len(df) == 0:
            pass
    
    # feature
    for day in [1,2,3,4,5,6]:
        df[f'{day}days_before_close'] = df['Close'].shift(periods=day+1)
    
    df = df.iloc[10:]
    df_model = df[['Open', 'High', 'Low', 'Close', f'1days_before_close', f'2days_before_close', f'3days_before_close', f'4days_before_close', f'5days_before_close', f'6days_before_close']]
    data_all = pd.DataFrame(df_model.values.flatten())
    data_all = np.array(data_all).astype(float)
    scaler = MinMaxScaler()
    data_all = scaler.fit_transform(data_all)
    data = []
    sequence_length = len(df_model.columns) # Feature Number
    for i in range(int(len(data_all) / sequence_length)):
        data.append(data_all[i*sequence_length:(i+1)*sequence_length])
    test_x = np.array(data).astype('float64')
    test_x_last = np.array([test_x[-1]])
    predict = models[0].predict(test_x)
    predict = np.reshape(predict, (predict.size, ))
    predict = scaler.inverse_transform([[i] for i in predict])
    area_data2 = predict
    labels = df.index.astype(str).str[:10].tolist()
    area_data1 = df['Close'].values.tolist()
    for day in [1,2,3,4,5]:
        labels.append(f'{day} Days Predict')
    for day in [1,2,3,4]:
        predict = models[day].predict(test_x_last)
        predict = np.reshape(predict, (predict.size, ))
        predict = scaler.inverse_transform([[i] for i in predict])
        area_data2 = np.append(area_data2, predict)
    
    area_data2 = np.around(area_data2.flatten(), 1).tolist()
    labels.pop(0)
    area_data1.pop(0)
    bar_data = df['Volume'].values.tolist()
    labels = str(labels).replace("'", "").replace("[", "").replace("]", "")
    area_data1 = str(area_data1).replace("[", "").replace("]", "")
    area_data2 = str(area_data2).replace("[", "").replace("]", "")
    bar_data = str(bar_data).replace("[", "").replace("]", "")
    return labels, area_data1, area_data2, bar_data

get_data('2330')

def get_twii_data():
    df = yf.Ticker('^TWII').history(start=start, end=today)
    for day in [1,2,3,4,5,6]:
        df[f'{day}days_before_close'] = df['Close'].shift(periods=day+1)
    
    df = df.iloc[10:]
    df_model = df[['Open', 'High', 'Low', 'Close', f'1days_before_close', f'2days_before_close', f'3days_before_close', f'4days_before_close', f'5days_before_close', f'6days_before_close']]
    data_all = pd.DataFrame(df_model.values.flatten())
    data_all = np.array(data_all).astype(float)
    scaler = MinMaxScaler()
    data_all = scaler.fit_transform(data_all)
    data = []
    sequence_length = len(df_model.columns) # Feature Number
    for i in range(int(len(data_all) / sequence_length)):
        data.append(data_all[i*sequence_length:(i+1)*sequence_length])
    test_x = np.array(data).astype('float64')
    test_x_last = np.array([test_x[-1]])
    predict = models[0].predict(test_x)
    predict = np.reshape(predict, (predict.size, ))
    predict = scaler.inverse_transform([[i] for i in predict])
    area_data2 = predict
    labels = df.index.astype(str).str[:10].tolist()
    area_data1 = df['Close'].values.tolist()
    for day in [1,2,3,4,5]:
        labels.append(f'{day} Days Predict')
    for day in [1,2,3,4]:
        predict = models[day].predict(test_x_last)
        predict = np.reshape(predict, (predict.size, ))
        predict = scaler.inverse_transform([[i] for i in predict])
        area_data2 = np.append(area_data2, predict)
    
    area_data2 = np.around(area_data2.flatten(), 1).tolist()
    labels.pop(0)
    area_data1.pop(0)
    # df = pd.read_csv('Stock_Profile.csv', index_col=0)
    # df = df[['Name', 'News size']]
    # df = df.sort_values(by=['News size'], ascending=False)[:10]
    # bar_labels = df['Name'].values.tolist()
    # bar_data = df['News size'].values.tolist()
    # labels = str(labels).replace("'", "").replace("[", "").replace("]", "")
    # area_data1 = str(area_data1).replace("[", "").replace("]", "")
    # area_data2 = str(area_data2).replace("[", "").replace("]", "")
    # bar_labels = str(bar_labels).replace("'", "").replace("[", "").replace("]", "")
    # bar_data = str(bar_data).replace("[", "").replace("]", "")
    bar_labels = "2021-02-01, 2021-02-02, 2021-02-03, 2021-02-04, 2021-02-05, 2021-02-06, 2021-02-07, 2021-02-08, 2021-02-09, 2021-02-10, 2021-02-11, 2021-02-12, 2021-02-13, 2021-02-14, 2021-02-15, 2021-02-16, 2021-02-17, 2021-02-18, 2021-02-19, 2021-02-20"
    bar_data = "165, 144, 141, 125, 118, 113, 112, 111, 107, 101, 99"
    return labels, area_data1, area_data2, bar_labels, bar_data

get_twii_data()